In [ ]:
import socket
import sys
import requests
import requests_oauthlib
import json

import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
import cv2
import keras

import tensorflow as tf
from keras.applications import VGG16,ResNet50,InceptionV3, MobileNetV2
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import urllib.request

import time


In [ ]:
global counter
counter = 1
filelist = [ f for f in os.listdir('./text_file')]
for f in filelist:
    os.remove(os.path.join('./text_file', f))
filelist = [ f for f in os.listdir('./picture')]
for f in filelist:
    os.remove(os.path.join('./picture', f))

In [ ]:
# set base_dir & model &weights path
BASE_DIR = ''
WEIGHTS_DIR = os.path.join(BASE_DIR, 'MobileNetV2_weights_5_10_fc3.h5')
# MODEL_DIR = os.path.join(BASE_DIR, 'MobileNetV2_model_5_10.h5')
CLASS_NAME = ['suit_adult','suit_none', 'suit_all']
TARGET_SHAPE = 224

In [ ]:
# 1. compile model, load weights
conv_base2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(TARGET_SHAPE, TARGET_SHAPE, 3))

In [ ]:
# define model
model2 = Sequential()
model2.add(conv_base2)
model2.add(Flatten())
model2.add(Dropout(0.5))
model2.add(Dense(64, activation=tf.nn.relu))
# model2.add(Dropout(0.4))
model2.add(Dense(32, activation=tf.nn.relu))
model2.add(Dense(32, activation=tf.nn.relu))
model2.add(Dense(3, activation='softmax'))

# model2.summary()
conv_base2.trainable=False

model2.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# load weights
model2.load_weights(WEIGHTS_DIR)

In [ ]:
def predict_img(model2, CLASS_NAME, img):
    # predict a image, return its class name
    img = cv2.resize(img, (TARGET_SHAPE, TARGET_SHAPE))
    img_pred = np.expand_dims(img,axis = 0)
    pred = np.argmax(model2.predict(img_pred))
    return CLASS_NAME[pred], pred

In [ ]:
# Replace the values below with yours
"""
CONSUMER_KEY = 'GpptFmNR9usFQ9AlxXenjH33C'
CONSUMER_SECRET = 'pBVjChbX97GpLpAQG3zm7HTPP34Ui5EbZfOUmCklr0f73WgDWH'
ACCESS_TOKEN = '715011126598078464-V72WAPYEI0m1UwjILwpYyM4EWIvFm3U'
ACCESS_TOKEN_SECRET = 'WCIFP0Bolszk6y7GCPLJ2rZ8BsK1OdbyMWdC51ezNK8xx'
"""

CONSUMER_KEY = "gogcVeWEFKKbL2ukNeApLAdOm" 
CONSUMER_SECRET= "dC10b43ZxblWn1JyRgyJ1J9HtW8BI3P6nAyF6rxnVqyMQE298C"
ACCESS_TOKEN = "1119010974315503616-jMb6tt2UndlpltbptiNFvfggHWnkWm"
ACCESS_TOKEN_SECRET = "NagNb7QdqTwnwpfEpleHjqKfJJYqmEKXByXpsKFN2pkUf"

my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
def common_list(list1, list2):
    list1 = [item.lower() for item in list1]
    list2 = [item.lower() for item in list2]
    l1_set = set(list1)
    for item in list2:
        if item in l1_set:
            return True, item
    return False, None
hashtags = ['pussy', 'cumslut', 'gangbang', 'nake', 'naked', 'dildo',  'webcams', \
           'nudes', 'freenudes', 'xxx', 'camgirl', 'amateur', \
           'squirt', 'nude', 'cumtribute', 'fuckme', "porn",\
           'fetish',  'fisting', 'massage', 'milf', \
           'bbw', 'blowjob', 'creampie', 'orgasm', \
           "cum","lesbian", "milf", "tit", 'tits', \
           "cock","hentai","anal",\
           "whore", "bitch", "lonely", \
           "nsfw", "sex", "sexy", "horny", 'cosplay', 'ass', "fuck", "hardcore"]     

In [ ]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [ ]:
def common(x, y):
    for i in x:
        if i in y:
            return True
    return False
        
def send_tweets_to_spark(http_resp, tcp_connection):
    global counter
    time_list = []
    time_step = 0
    image_num = 0
    filter_num = 0
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            
            hashtags_raw = full_tweet["entities"]["hashtags"]
            hashtags_got = list()
            for hashtag in hashtags_raw:
                hashtags_got.append(hashtag["text"].lower())
                
            # if any hashtag match happens
#             if not common(hashtags, hashtags_got):
#                 continue
            s = tweet_text.split(' ')
            flag, data = common_list(s, hashtags)
            if "extended_entities" in full_tweet:
#             if "extended_entities" in full_tweet and flag == True:
                time_list += [time.time()]
                print('Keyword:', data)
                if hashtags_got:
                    print('Hashtag:', hashtags_got)
                else:
                    print('Hashtag: no hashtag')
                
                print('Tweet text:\n', tweet_text)
                if "media" in full_tweet["extended_entities"]:
                    f = open('./text_file/'+str(counter)+'.txt', 'w+')
                    f.writelines('keyword   ')
                    f.writelines(str(data))
                    f.writelines('\n')
                    f.writelines('hashtag   ')
                    f.writelines(str(hashtags_got))
                    f.writelines('\n')
                    f.writelines('text   ')
                    f.writelines(tweet_text)
                    f.writelines('\n')
                    j = 1
                    picture_list = []
                    result_list = []
                    tweet_image = []
                    for i in range(len(full_tweet["extended_entities"]["media"])):
                        tweet_image += [full_tweet["extended_entities"]["media"][i]["media_url"]]
                        picture_list += [str(counter)+'_'+str(j)+'.jpg']
                        urllib.request.urlretrieve(full_tweet["extended_entities"]["media"][i]["media_url"], './picture/'+str(counter)+'_'+str(j)+'.jpg')
                        img = cv2.imread('./picture/'+str(counter)+'_'+str(j)+'.jpg')
                        res_class, _ = predict_img(model2, CLASS_NAME, img)
                        result_list += [res_class]
                        j += 1
                        image_num += 1
                        if res_class == 'suit_adult' or res_class == 'suit_none':
                            filter_num += 1
                    
                    f.writelines('picture')
                    print("Picture url:", end = '   ')
                    for k in range(len(picture_list)):
                        f.writelines('   ')
                        f.writelines(str(picture_list[k]))
                        print(str(tweet_image[k]), end = '   ')
                    f.writelines('\n')
                    print('\n', end = '')
                    
                    f.writelines('result')
                    print("Result:", end = '   ')
                    for k in range(len(result_list)):
                        f.writelines('   ')
                        f.writelines(str(result_list[k]))
                        print(str(result_list[k]), end = '   ')
                    f.writelines('\n')
                    print('\n', end = '')
                    counter += 1
                    
                    for i in range(time_step, len(time_list)):
                        total_time = time.time() - time_list[i]
                        if total_time > 10:
                            os.remove('./text_file/' + str(i+1) + '.txt')
                            for loopp in range(1, 5):
                                try:
                                    os.remove('./picture/' + str(i+1) + '_' + str(loopp) + '.jpg')
                                except:
                                    pass
                        else:
                            time_step = i
                            break
                    
                    print('Total picture numbers:', image_num)
                    print('Total filter numbers:', filter_num)
                    print('ratio:', str(round(filter_num/image_num, 1)))
                    f2 = open('counter.txt', 'w')
                    f2.write(str(image_num))
                    f2.write('\t')
                    f2.write(str(filter_num))
                    f2.write('\t')
                    f2.write(str(round(filter_num/image_num, 1)))
                    f2.close()
                    
                    print ("end------------------------------------------")
                    tcp_connection.send(line)
                else:
                    print("picture url: No image")
                    print ("end------------------------------------------")
        except:
            raise
            e = sys.exc_info()[0]
            print("Error: ", e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

In [ ]:
! lsof -i:9010

In [ ]:
#! kill 

In [ ]:
print(time_list[0])